In [190]:
import keras
from keras import backend as K
import numpy as np
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

In [191]:
# predictions on probe
probe_predictions = [#"data/kevin/probe/out_probe_svd_1.92.dta",
                     # "data/misc/all_threes_probe.dta",
                     "data/frank/SVDPP_prediction_probe4-04.dta",
                     #"data/kevin/probe/out_probe_knn_2.dta",
                     "data/james/probe/SVD_probe5-08.dta",
                     "data/james/probe/Time_SVDpp_probe6-50.dta"
                    ]                                    
# predictions on quiz
qual_predictions = [#"data/kevin/qual/out_qual_svd_1.92.dta",
                    #"data/misc/all_threes_qual.dta",
                    "data/frank/SVDPP_prediction_qual4-04.dta",
                    #"data/kevin/qual/out_qual_knn_2.dta"
                    "data/james/qual/SVD_qual5-08.dta",
                    "data/james/qual/Time_SVDpp_qual6-50.dta"
                    ]
num_models = 0

In [192]:
num_models = 0
if len(probe_predictions) == len(qual_predictions):
    num_models = len(probe_predictions)
    print("GOOD TO GO")
else:
    print("WE HAVE A PROBLEM")

GOOD TO GO


In [193]:
# load predictions on probe
X_train = []
for preds in probe_predictions:
    X_train.append(np.loadtxt(preds).astype(float))
    
# load probe (just the ratings)
probe = np.loadtxt('probe_ratings.dta').astype(int)

In [194]:
# load predictions on quiz
X_test = []
for preds in qual_predictions:
    X_test.append(np.loadtxt(preds).astype(float))

In [195]:
# load probe (user, movie, time)
probe_etc = np.loadtxt('probe.dta', usecols = (0,1,2)).astype(int)
print(probe_etc)
# load qual (user, movie, user, time)
qual_etc = np.loadtxt('qual.dta', usecols = (0,1,2)).astype(int)
print(qual_etc)

[[     1  14708   2160]
 [     1  16933   2160]
 [     2   3912   2039]
 ...
 [458292  13157   2211]
 [458292  14582   2184]
 [458293   8578   2224]]
[[     1   4624   2160]
 [     1   7218   2160]
 [     1   8346   2160]
 ...
 [458293  12636   2232]
 [458293  15905   2224]
 [458293  17425   2236]]


In [196]:
# Transpose the matrices so they have dimension NUM_QUAL/NUM_PROBE x num_models
X_train = np.array(X_train).T
X_train = np.concatenate((X_train, probe_etc), axis = 1)
print(X_train.shape)
X_test = np.array(X_test).T
X_test = np.concatenate((X_test, qual_etc), axis = 1)
print(X_test.shape)

(1374739, 6)
(2749898, 6)


In [202]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [219]:
# making the neural network model
def baseline_model():
    # create model
    model = Sequential()
    #model.add(Dense(13, input_dim = num_models, kernel_initializer='normal', activation='relu'))
    model.add(Dense(45, input_dim = num_models + 3, activation='relu'))
#     model.add(Dropout(0.1))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation = 'linear'))
#     model.add(Dropout(0.1))
    model.add(Dense(1))
    model.compile(loss= root_mean_squared_error, metrics=['acc'], optimizer='adam')
    return model

In [220]:
model = baseline_model()
model.fit(X_train, probe, epochs=7, batch_size = 200)
ratings = model.predict(X_test)

Epoch 1/7
1374739/1374739 [==============================] - 92s 67us/step - loss: 48.6318 - acc: 0.1385
Epoch 2/7
1374739/1374739 [==============================] - 43s 31us/step - loss: 1.1600 - acc: 0.3068
Epoch 3/7
1374739/1374739 [==============================] - 63s 46us/step - loss: 1.1486 - acc: 0.3094
Epoch 4/7
1374739/1374739 [==============================] - 43s 31us/step - loss: 1.1178 - acc: 0.3200
Epoch 5/7
1374739/1374739 [==============================] - 51s 37us/step - loss: 1.0604 - acc: 0.3456
Epoch 6/7
1374739/1374739 [==============================] - 51s 37us/step - loss: 1.0414 - acc: 0.3568
Epoch 7/7
1374739/1374739 [==============================] - 67s 49us/step - loss: 1.0617 - acc: 0.3500


In [221]:
ratings = np.clip(ratings, 1, 5)
ratings = ratings.ravel()
print(ratings)

[3.4660058 3.3558054 4.507613  ... 3.3097897 3.3097897 3.3097897]


In [222]:
f = open("output/new_nn.dta", "w+")
for i in range(len(ratings)):
    if (i % 1000000 == 0):
        print(i)
    f.write(str(ratings[i]) + '\n')
f.close()

0
1000000
2000000
